- 撰稿人：許恆修

# 安裝 & 基本用法

Python 第三方 Library Gradio 快速上手
- python 版本要求 3.7 以上

In [ ]:
!pip install --quiet gradio

檢視 Python 版本

In [ ]:
!python -V

檢視 Gradio 版本

In [ ]:
!pip show gradio

---

#建立聊天機器人

在人工智慧（AI）領域中，**聊天機器人**（Chatbot）已成為大型語言模型（LLMs, Large Language Models）最受歡迎的應用之一。借助 **Gradio**，你可以輕鬆打造 LLM 的互動示範應用並與使用者共享，或直接透過直覺式聊天介面進行測試。本教學將示範如何透過 `gr.ChatInterface()` 快速構建聊天機器人，以及如何進一步客製化你的系統。

## 結構介紹

讓我們先整體說明要完成的內容有哪些，可以看到下圖列表結構。

## 認識 gr.ChatInterface()

1. 高階抽象，一行程式碼打造聊天機器人
gr.ChatInterface() 是 Gradio 提供的高階工具，能讓你用最少的 Python 程式碼建立一個全功能的對話介面。它可輕易擴充至支援多模態（multimodal）聊天機器人，或在更複雜的需求下進行自訂化。

- message：使用者當前輸入的文字（str）
- history：過往聊天歷史（list），包含 role 與 content 等欄位，可能還包含其他自訂鍵值對
一個典型的 history 會長這樣：

```python
[
    {"role": "user", "content": "What is the capital of France?"},
    {"role": "assistant", "content": "Paris"}
]
```
你的 Chatbot 只需要回傳一個 str，表示基於這些對話內容所生成的回應。

## 定義聊天函式

### 範例 1：隨機回應「Yes」或「No」

以下為最基本的聊天函式，它不會參考使用者輸入或對話歷史，只是隨機回應「Yes」或「No」：

```python
import random
def random_response(message, history):
    return random.choice(["Yes", "No"])
```

接著，我們只需將此函式與 gr.ChatInterface() 綁定，並設定 type="messages"（強烈建議這麼做），就能立即執行：

```python
import gradio as gr

gr.ChatInterface(
    fn=random_response,
    type="messages"
).launch()
```



###範例 2：依序「同意」或「不同意」的簡單邏輯

稍微進階一點的例子：根據先前對話中機器人回應的次數，交替回傳「Yes...」或「I don't think so」。

```python
import gradio as gr

def alternatingly_agree(message, history):
    # 計算助理出現的次數，來判斷要「同意」或「不同意」
    if len([h for h in history if h['role'] == "assistant"]) % 2 == 0:
        return f"Yes, I do think that: {message}"
    else:
        return "I don't think so"

gr.ChatInterface(
    fn=alternatingly_agree,
    type="messages"
).launch()
```

###串流（Streaming）聊天機器人

若想要看到「字逐字顯示」的效果，可以利用 yield 回傳一連串局部回應，每個新的回應會覆蓋掉先前的訊息，達到串流效果。範例如下：


```python
import time
import gradio as gr

def slow_echo(message, history):
    for i in range(len(message)):
        time.sleep(0.3)
        yield "You typed: " + message[: i+1]

gr.ChatInterface(
    fn=slow_echo,
    type="messages"
).launch()
```

當使用者點擊「Submit」後，按鈕會變成「Stop」，可隨時停止串流輸出。

### 額外的輸入 (Additional Inputs)

有時候你需要 Chatbot 以外的其他輸入，如「系統提示」（system prompt）或「回應長度控制」等，可以透過 additional_inputs 參數達成。此參數可接受單一元件或多個元件的清單。

```python
import gradio as gr
import time

def echo(message, history, system_prompt, tokens):
    response = f"System prompt: {system_prompt}\nMessage: {message}."
    for i in range(min(len(response), int(tokens))):
        time.sleep(0.05)
        yield response[: i + 1]

demo = gr.ChatInterface(
    echo,
    type="messages",
    additional_inputs=[
        gr.Textbox("You are helpful AI.", label="System Prompt"),
        gr.Slider(10, 100),
    ],
)

demo.launch()

```

###預設回覆（Preset Responses）

想在對話回應中提供「選項按鈕」？可於回傳字典中加入 options，例如：

```python
import gradio as gr

example_code = '''
Here's the code I generated:

def greet(x):
    return f"Hello, {x}!"

Is this correct?
'''

def chat(message, history):
    if message == "Yes, that's correct.":
        return "Great!"
    else:
        return {
            "role": "assistant",
            "content": example_code,
            "options": [
                {"value": "Yes, that's correct.", "label": "Yes"},
                {"value": "No"}
            ]
        }

demo = gr.ChatInterface(
    chat,
    type="messages",
    examples=["Write a Python function that takes a string and returns a greeting."]
)

if __name__ == "__main__":
    demo.launch()

```

## 總結
透過 gr.ChatInterface()，你能在極短的時間內建立一個功能完備的聊天機器人，並根據需求輕鬆地：

串接多模態功能，允許文字、圖片、音訊等多種形式的輸入
增加額外的輸入輸出，如「系統提示」、「程式碼區塊」或「滑桿參數」
客製化介面、增加範例按鈕或預設回覆選單
以串流方式呈現回應，提供更逼真的對話體驗
不論你是想製作簡單的示範原型，還是部署複雜的企業級應用，Gradio 都能協助你快速上手，並提供高度可擴充的客製化能力，將 LLM 潛能充分展現給使用者。

# 練習題：為 Chatbot 加入「系統提示」與「字數限制」的額外輸入

### **題目描述**
請嘗試在原有的 Chatbot 基礎上，新增兩個額外輸入：
1. 一個 `Textbox` 作為「系統提示」（System Prompt），此文字內容會影響整體對話風格或角色設定。  
2. 一個 `Slider` 用來控制 Chatbot 回應的最大字數（或 Token 數）。

建立此多輸入 Chatbot 時，請參考上方材料或自行翻閱 Gradio 文件中的 `additional_inputs` 章節，並設計一個函式能夠在產生回應時依照使用者的「系統提示」與「字數限制」執行。

### **情境**
想像你在開發一個專業諮詢聊天機器人，你需要在每次對話前指定諮詢領域或角色（例如：心理諮商師、理財顧問、健身教練），同時希望能在系統內控管回應長度，避免過度冗長或不符合風格。這時就可以利用「額外輸入」的概念為 Chatbot 添加更多動態參數。

### **解答 (範例)**
```python
import gradio as gr
import time

def role_based_responder(message, history, role_prompt, max_length):
    """
    - message: 使用者輸入的文字
    - history: 過往對話紀錄
    - role_prompt: 系統提示，決定 Chatbot 的角色或風格
    - max_length: 回應的最大字數
    """
    # 將系統提示與使用者訊息整合，形成 Chatbot 回應的基礎
    combined_prompt = f"【系統提示】{role_prompt}\n【使用者訊息】{message}\n"
    
    # 建立一個假設的回應（在真實情況中可串接 LLM API）
    full_response = f"你好！我是你設定的角色：{role_prompt}\n關於你的問題：{message}\n以下是我的建議..."

    # 使用 yield 做串流回應，並根據 max_length 控制輸出長度
    truncated_response = full_response[:int(max_length)]
    for i in range(len(truncated_response)):
        time.sleep(0.05)  # 模擬模型思考時間
        yield truncated_response[:i+1]

demo = gr.ChatInterface(
    fn=role_based_responder,
    type="messages",
    additional_inputs=[
        gr.Textbox(
            value="請扮演一位溫暖且耐心的心理諮商師",
            label="系統提示 (Role Prompt)"
        ),
        gr.Slider(10, 200, step=10, value=100, label="回應字數上限 (Max Length)")
    ],
    title="角色扮演與字數控制 Chatbot",
    description="依照系統提示扮演不同角色，並用滑桿控制回應字數長度"
)

demo.launch()
```

> **完整要求**：
> 1. 建立一個具備兩個額外輸入的 Chatbot：  
>   - `Textbox`：擔任「系統提示」，可以讓使用者修改角色設定。  
>   - `Slider`：控制最大回應字數（或 Token 數）。  
> 2. 在回應函式中，需同時讀取文字輸入、系統提示，以及對話紀錄（`history`）。  
> 3. 以串流（`yield`）方式回傳 Chatbot 的回答，確保每次輸出的累積不超過 `max_length`。  
> 4. 在界面上提供標題與描述，協助使用者理解該 Chatbot 的用途。  
> 5. 若有疑問，請回頭查看上方材料關於 `additional_inputs` 與串流機制（Streaming）的說明，或參考 Gradio 官方文件。
```